# Provision via FLARE Dashboard

As mentioned in the [introduction to NVIDIA FLARE provisioning](../04.0_introduction/provision.ipynb), one way of performing provisioning is to use the [FLARE Dashboard](https://nvflare.readthedocs.io/en/2.6/user_guide/dashboard_api.html).

[FLARE Dashboard](https://nvflare.readthedocs.io/en/2.6/user_guide/dashboard_api.html) provides an intuitive webUI to collect information about clients and users from different organizations, and to provision a federated project by generating startup kits for users to download.

FLARE Dashboard simplifies the process of data collection. Users can register to join the project and provide their own information, and then download their own startup kits once the project admin has approved the registration. All the project information can be managed online with provisioning done on the fly.

![Dashboard](images/dashboard.png)


# Start, Configure and Stop the FLARE Dashboard

NVIDIA FLARE provides a CLI tool `nvflare dashboard` to start, stop and configure the Dashboard:

> **Note:** Running Dashboard requires Docker. You have to ensure your system can pull and run Docker images. The initial docker pull may take some time depending on your network connection.
> 

In [ ]:
!nvflare dashboard -h

There are many options with this CLI tool, allowing you to customize the behavior of the Dashboard.

### Starting the Dashboard

To start Dashboard, run `nvflare dashboard --start`.

The Dashboard Docker will detect if the database is initialized. If not, it will ask for the project_admin email address and will generate a random password. You would need to log in with this random password to finish setting up the project in Dashboard once the system is up and running. The project_admin can change his/her password in the Dashboard system after logging in.

Note that for the first time, it may take a while to download the nvflare Docker image. After pulling the image, you should see output similar to the following:
```
Launching nvflare/nvflare
Dashboard will listen to port 443
/path_to_folder_for_db on host mounted to /var/tmp/nvflare/dashboard in container
No additional environment variables set to the launched container.
Dashboard container started
Container name nvflare-dashboard
id is 3108eb7be20b92ab3ec3dd7bfa86c2eb83bd441b4da0865d2ebb10bd60612345
```

FLARE Dashboard is hosted through port `443`, so that you can access it through `localhost:443` in a web browser, if you are on the same machine. You can also remotely access it after configuring proper port forwarding.

### Configuring the Dashboard

It is recommended to set a passphrase to protect the private key of the root CA by using the `--passphrase` option when starting a dashboard. Once it’s set, you have to provide the same passphrase everytime you restart Dashboard for the same project.

You can use the `--folder` option to customize the working directory for the Dashboard. If you would like to start a new project, please remove the `db.sqlite` file in current working directory (or the directory set with the `--folder` option). Dashboard will start from scratch and you can provide a project admin email address and get a new password for the project_admin.

The Dashboard will also check the certification folder inside current the working directory (or directory specified by the `--folder` option) to load `web.crt` and `web.key`. If those files exist, Dashboard will load them and run as an HTTPS server. If Dashboard does not find both of them, it runs as HTTP server. In both cases, the service listens to port `443`, unless the `--port` option is used to specify a different port. Dashboard will run on `0.0.0.0`, so by default it should be accessible on the same machine from `localhost:443`. To make it available to users outside the network, port forwarding and other configurations may be needed to securely direct traffic to the machine running the Dashboard.


### Stopping the Dashboard

To stop the running Dashboard, run `nvflare dashboard --stop`.

# Example: Provision a Federated Project With Dashboard

## 1. Create & Start a Dashboard

Let's first go ahead and start a Dashboard, by typing the following command in a terminal:

```bash
nvflare dashboard --start --port 567 --folder ./dashboard-wd --passphrase 12345 
```

Here we use the following options to configure the Dashboard start-up:
- Use the `--port` option to explicitly set the port to `567`
- Use the `--folder` option to specify a working directory for the Dashboard
- Use the `--passphrase` option to specify a passphrase for encryption

When you create a Dashboard for the first time, you will be prompt to enter an admin email, which will be the login for the admin user. Choose an email address for your own. In this example, I will be using `admin@nvidia.com`.

When the Dashboard is successfully created and started, you will see console output similar to the following:
```
Please provide project admin email address.  This person will be the super user of the dashboard and this project.
admin@nvidia.com
generating random password
Project admin credential is admin@nvidia.com and the password is **wGNrI56s**
Current dashboard container image is nvflare/nvflare:2.5.0+161.ge9c5749e.dirty, but requesting to use nvflare/nvflare:latest.  Use it at your own risk.
Pulling nvflare/nvflare:latest, may take some time to finish.
Launching nvflare/nvflare:latest
Dashboard will listen to port 443
/home/zhijinl/github/NVFlare/examples/tutorials/self-paced-training/part-2_federated_learning_system/chapter-4_setup_federated_system/04.2_provision_via_dashboard/./dashboard-wd on host mounted to /var/tmp/nvflare/dashboard in container
environment vars set to {'NVFL_DASHBOARD_PP': '12345', 'NVFL_CREDENTIAL': 'admin@nvidia.com:wGNrI56s'}
Dashboard container started
Container name nvflare-dashboard
id is 7f15a879fc75d6cd797591cd840840ebafd6b194bb86dda56efcbc36d719e773
```

Notice the highlighted password for the admin user. Note it down somewhere, you will need it when logging into the Dashboard as admin. You will have the option to change the admin password once logged in.

Open a web browser and navigate to `localhost:443`, you will see a landing page with empty project information, similar to the following:

![Landing Page](images/landing-page.png)

## 2. Users and Roles

Before moving on, it is necessary to understand FLARE Dashboard's distinct definitions of different users and their roles.

- **Project Admin**: project administrator is responsible for overall site management from setting up the project to approving the users and client sites after inviting users to sign up. The Project Admin can also download startup kits for the server(s) and overseer (if HA mode is enabled).

- **Org Admin**: Users of the role org admin have the ability to specify the name and resource specifications for client sites and then download startup kits for each client site after it is approved.

- **Member and Lead**: users with role member or lead can register a user account and then download their startup kit to connect to the FLARE console, once their account is approved.

In this example, we will demonstrate the following:
- Setting up a project with project admin
- Org admin user sign-up
- Configuring client sites with org admin
- Downloading startup kits for different participants in Dashboard

For a more detailed walkthrough of the Dashboard UI and usage, please refer to the [documentation](https://nvflare.readthedocs.io/en/2.6/user_guide/dashboard_ui.html#member-and-lead-user-experience).


## 3. Set Up the Project

Now, log in the Dashboard with your admin email and password, and let's start setting up the project. 

As the project administrator, you will have the most priviledged right to manage the whole federated project, including for instance:
- Setting up the project information
- Configuring server information
- Making the project public for users to sign up
- Approving user sign-up
- etc.

Let's first set up the project information, by going to the "Project Configuration" tab, and entering the project's short name, title and description. When you are ready, you can click on "Make Project Public" button. After the project becomes public, new users can sign up and join the project.

![Project Setup](images/project-setup.png)

Next, let's set up the server information, by going to the "Server Configuration" tab, and enter the DNS information for the server. You can optionally set up a back-up server and overseer to enable high-availability mode.

![Server Setup](images/server-setup.png)

After the project and server information is set up, we can freeze the project, by going to the "Project Home" tab, and click on the "Freeze Project" button. Note that once the project is freezed, you can no longer modify its information.

![Freeze](images/freeze-project.png)


## 4. User Sign-up

Once a federated project is made public, new users can sign up and join the project. On the landing page, you will see a new "Signup" button. 

![Signup](images/signup.png)

Click on it, and you will be brought to the new user sign-up page. Let's create a org admin user with email address "org_admin@nvidia.com":

![New User](images/new-user.png)

![Org Admin](images/org-admin.png)

After sign-up, the project admin would need to approve the sign-up before the org admin can log into the project. Let's go ahead and log in as the project admin again and approve the org admin user sign-up.

![Approve User](images/approve-user.png)

## 5. Set Up Client Sites

Log into the project as the org admin. As an org admin, you are able to add client sites to the federated project.

![Add Site](images/add-site.png)

Let's go ahead and add two clients: `site-1` and `site-2` to the project. You can specify the GPU resources for each site. After adding the clients, log in as the project admin again and approve client sites.

![Approve Clients](images/approve-clients.png)

After the client sites are approved, you will see them appear as properly set up in the org admin's view.

![Clients Approved](images/clients-approved.png)

### 6. Download Startup Kits

Once the project, the server and the clients are all set up, we can proceed to download the startup kits. Besides downloading their own startup kits to connect to the FLARE Console, the project admin can download the server startup kit:

![Project Startup Kit](images/admin-startup-kit.png)

And the org admin can download & distribute the startup kits for clients:

![Org Startup Kits](images/org-startup-kits.png)


**That's it, we have created a Dashboard, and walked through how to propoerly provision & set up a project with it!**

# What's Next

Next, we will learn how to perform preliminary checks beforing starting a live federated system with [preflight check](../04.3_preflight_check/preflight_check.ipynb).